In [1]:
import pandas as pd
import numpy as np
from pyhive import hive
from datetime import datetime,timedelta
from datetime import timedelta
import datetime
import gc
import re
import json
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)

In [2]:
final = pd.read_csv('raw_data.csv')

In [3]:
final

,Account No,Alternate Acc No,allocationId,POS,TOS,DPD,product,institutionId,identifierKey,allocationAccountType,phone,DOB,pincode,accountRetentionStatus,offerAmount,offerExpiryDate,leadRank,customerAddress,CustomerName,employerName,mobileMatch,City,State,@Status,Base Mode,LM Retain Status,Lead ID,Cust Id,Int/Ext,Removed/Escalation,Upload Count,Priority,Feedback,Retain/Rel- CM,Internal No,Account No.1,Revisit,Loan Enquiry,Unnamed: 38
0,7456962,4375510093999008,NaN,1873.01,7475.88,3503,CARDS,ICICI_BANK,1.0,WRITEOFF,9.370202e+09,0.0,411014,FRESH,NaN,NaN,607,FL NO 05 WHITE HOUSE VADGAON SHERI BOLLYWOOD ...,PRITAM RAMESH PATIL,Not Available,0.0,PUN,Not available,Retain,Gurgaon,Retain,1360264346,2885473,Internal,NaN,1.0,Priority 4,NaN,NaN,8237113447,4375510093999008,NaN,NaN,NaN
1,7458063,4375510286714008,NaN,7208.64,40129.48,2775,CARDS,ICICI_BANK,1.0,WRITEOFF,9.820607e+09,0.0,400051,FRESH,NaN,NaN,605,207/CMC RESIDENCY BANDRA EAST R-11 BANDRA KURL...,MSPRANAV ANAND,Not Available,0.0,MUM,Not available,Fresh,Gurgaon,Fresh,1361057024,6661749,Internal,NaN,1.0,No Priority,NaN,NaN,0x2a,4375510286714008,NaN,NaN,NaN
2,7458770,4375510317187000,NaN,26867.32,52625.09,2761,CARDS,ICICI_BANK,1.0,WRITEOFF,7.569289e+09,0.0,400093,FRESH,NaN,NaN,604,BLDG NO-R6 M M R D A PLOT NO 206 POONAM NAGAR ...,PARUSURAMUDU KURUVA,Not Available,0.0,MUMBAI,MAH,NaN,Gurgaon,Fresh,1361529785,25069076,Internal,NaN,1.0,No Priority,NaN,NaN,0x2a,4375510317187000,NaN,NaN,NaN
3,7459213,5239512553836004,NaN,44793.10,76165.08,2354,CARDS,ICICI_BANK,1.0,WRITEOFF,9.780263e+09,0.0,145001,FRESH,NaN,NaN,604,"HOUSE NUM:62 OPP STREET KALIMATHA MANDIR , PAT...",RAVI KUMAR,Not Available,0.0,GURDASPUR,PUN,NaN,Gurgaon,Fresh,1361587425,41419080,Internal,NaN,1.0,No Priority,NaN,NaN,0x2a,5239512553836004,NaN,NaN,NaN
4,7460076,4375510094874002,NaN,37624.82,62066.58,2106,CARDS,ICICI_BANK,1.0,WRITEOFF,9.810600e+09,0.0,110044,FRESH,NaN,NaN,604,C 52 SAURABH VIHAR BADARPUR 1ST FLOOR XXX,RANI JHA,Not Available,0.0,DELHI,DEL,NaN,Gurgaon,Fresh,1361551828,4058260,Internal,NaN,1.0,No Priority,NaN,NaN,0x2a,4375510094874002,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516698,UVTNE00008503728,UVTNE00008503728,NaN,98774.00,491614.00,5444,CB,ICICI_BANK,1.0,WRITEOFF,9.324451e+09,0.0,400104,FRESH,NaN,NaN,1351.0,GALA MURSIMAYA SHANTIAARDWAR BHAGAT SINGH NAGAR,VIPUL MULJIBHAI GALA,Not Available,0.0,MUMBAI,NaN,NaN,Gurgaon,Retain,1361855126,19372954,Internal,NaN,1.0,No Priority,NaN,NaN,9920585390,UVTNE00008503728,NaN,NaN,NaN
516699,UVUJN00041401792,UVUJN00041401792,NaN,663570.50,317441.00,145,CB,ICICI_BANK,1.0,WRITEOFF,9.425023e+09,0.0,456443,FRESH,NaN,NaN,1351.0,NAGORI MOHALLA MAHIDPUR DIST UJJAIN,JAVED NAGORI,Not Available,0.0,MAHIDPUR,NaN,NaN,Gurgaon,Retain,0x2a,0x2a,Internal,NaN,0.0,Priority 5,NaN,NaN,9425022690,UVUJN00041401792,NaN,NaN,NaN
516700,UVUJN00042090659,UVUJN00042090659,NaN,201093.00,169714.00,380,CB,ICICI_BANK,1.0,WRITEOFF,9.425023e+09,0.0,456443,FRESH,NaN,NaN,1351.0,NAGORI MOHALLA MAHIDPUR DIST UJJAIN,JAVED NAGORI,Not Available,0.0,MAHIDPUR,NaN,NaN,Gurgaon,Retain,1360339141,40288687,Internal,NaN,1.0,Priority 4,NaN,NaN,9424878346,UVUJN00042090659,NaN,NaN,NaN
516701,WPAPG00034126033,WPAPG00034126033,NaN,12268.10,29141.23,1432,PL,ICICI_BANK,1.0,WRITEOFF,9.799482e+09,0.0,335701,RETAINED,NaN,NaN,612.0,JALOKI PADAMPUR SRI GANGANAGAR,SHANKAR LAL,Not Available,0.0,ANUPGARH,NaN,NaN,Gurgaon,Fresh,1360339888,72108831,Internal,NaN,1.0,No Priority,NaN,NaN,0x2a,WPAPG00034126033,NaN,NaN,NaN


In [4]:
final['phone'] = pd.to_numeric(final['phone'], errors = 'coerce')

In [5]:
final = final[['phone']]

In [6]:
final.phone = final.phone.astype(int)

In [7]:
final.rename(columns={'phone': 'mobilePhone'}, inplace=True)


In [8]:
final.dtypes

mobilePhone    int64
dtype: object

In [10]:
final.mobilePhone.nunique()

491087

In [9]:
int_col = ['mobilePhone','applicationCreditReportId','accountHolderTypeId','createdAt','reportDate','dateClosed',
        'openDate','dateReported','accountTypeId','accountStatusId','pincode','cityId','gender',
        'employmentTypeId','monthlyIncome','Age', 'score','accountStatusName','accountTypeName','paymentHistoryProfile','highCredit','creditLimitAmount','currentBalance','amountPastDue','dateofLastPayment']
scrub_col = ['mobilePhone','panId','accountHolderTypeId','createdAt','reportDate','dateClosed',
        'openDate','dateReported','accountTypeId','accountStatusId','pincode','cityId','gender',
        'employmentTypeId','monthlyIncome','Age', 'score','accountStatusName','accountTypeName','paymentHistoryProfile','highCredit','creditLimitAmount','currentBalance','amountPastDue','dateofLastPayment']
# /pb_analytics/analytics/acc_data/Using%20Hive%20Only/2022/05May
# date_list = ['Cibil_Scrub_31July23','Cibil_Scrub_30June23']
date_int = ['11Nov','Cibil_Scrub_29November23','10Oct','Cibil_Scrub_30October23','09Sep','Cibil_Scrub_26September23', '08Aug', 'Cibil_Scrub_29August23']
folder = ["/Bureau","/Internal"]
pans_data = pd.DataFrame()
mob = []
for bur in date_int:
#     if((bur == '12Dec') | (bur == '11Nov') | (bur == '10Oct') | (bur == '09Sep')):
#         year = 2022
#     else:
#         year = 2023
    print(bur)
    if bur.startswith('Cibil_Scrub'):
        print('on scrub')
        for f in folder:
            print(f)
            for i in range(1,80):
                    try:
                        path = "/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/"+str(bur)+str(f)+"/cibil_pt_0"+str(i)+".parquet.gzip"
                        print(path)
                        dd = pd.read_parquet(path,columns=scrub_col).drop_duplicates()
                        dd['mobilePhone'] = pd.to_numeric(dd['mobilePhone'], errors = 'coerce')
                        
#                         dd.drop_duplicates(subset = ['mobilePhone'], inplace = True)
                        dd = dd[dd.mobilePhone.isin(final['mobilePhone'])]
                        dd = dd.loc[~dd['mobilePhone'].isin(mob)]
                        mob = mob + dd['mobilePhone'].to_list()
                        pans_data = pans_data.append(dd)
                        print(pans_data['mobilePhone'].nunique())
                        del dd
                        gc.collect()
                    except:
                        pass
    else:
        for i in range(1, 34):
            try:
                path = '/pb_analytics/analytics/acc_data/Using Hive Only/2023/'+str(bur)+'/Cibil_'+str(i)+'.parquet.gzip'
                #path = '/pb_analytics/analytics/acc_data/Using Hive Only/2022/'+str(q)+'/Cibil_'+str(i)+'.parquet.gzip'
                print(path)
                dd = pd.read_parquet(path, columns = int_col).drop_duplicates()
                dd['mobilePhone'] = pd.to_numeric(dd['mobilePhone'], errors = 'coerce')
        #                     dd.drop_duplicates(subset = ['mobilePhone'],inplace = True)
                dd = dd[dd.mobilePhone.isin(final['mobilePhone'])]
                dd = dd.loc[~dd['mobilePhone'].isin(mob)]
                mob = mob + dd['mobilePhone'].to_list()
                pans_data = pans_data.append(dd)
                print(pans_data['mobilePhone'].nunique())
                del dd
                gc.collect()
            except:
                pass







11Nov
/pb_analytics/analytics/acc_data/Using Hive Only/2023/11Nov/Cibil_1.parquet.gzip
885
/pb_analytics/analytics/acc_data/Using Hive Only/2023/11Nov/Cibil_2.parquet.gzip
1669
/pb_analytics/analytics/acc_data/Using Hive Only/2023/11Nov/Cibil_3.parquet.gzip
3386
/pb_analytics/analytics/acc_data/Using Hive Only/2023/11Nov/Cibil_4.parquet.gzip
4966
/pb_analytics/analytics/acc_data/Using Hive Only/2023/11Nov/Cibil_5.parquet.gzip
6793
/pb_analytics/analytics/acc_data/Using Hive Only/2023/11Nov/Cibil_6.parquet.gzip
8410
/pb_analytics/analytics/acc_data/Using Hive Only/2023/11Nov/Cibil_7.parquet.gzip
10038
/pb_analytics/analytics/acc_data/Using Hive Only/2023/11Nov/Cibil_8.parquet.gzip
11711
/pb_analytics/analytics/acc_data/Using Hive Only/2023/11Nov/Cibil_9.parquet.gzip
13163
/pb_analytics/analytics/acc_data/Using Hive Only/2023/11Nov/Cibil_10.parquet.gzip
14617
/pb_analytics/analytics/acc_data/Using Hive Only/2023/11Nov/Cibil_11.parquet.gzip
15710
/pb_analytics/analytics/acc_data/Using Hiv

/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Bureau/cibil_pt_055.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Bureau/cibil_pt_056.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Bureau/cibil_pt_057.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Bureau/cibil_pt_058.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Bureau/cibil_pt_059.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Bureau/cibil_pt_060.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Bureau/cibil_pt_061.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Bureau/cibil_pt_062.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/Ex

/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Internal/cibil_pt_069.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Internal/cibil_pt_070.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Internal/cibil_pt_071.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Internal/cibil_pt_072.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Internal/cibil_pt_073.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Internal/cibil_pt_074.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Internal/cibil_pt_075.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29November23/Internal/cibil_pt_076.parquet.gzip
/pb_analytics/analytics/240/Sart

53164
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Bureau/cibil_pt_033.parquet.gzip
53230
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Bureau/cibil_pt_034.parquet.gzip
53296
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Bureau/cibil_pt_035.parquet.gzip
53381
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Bureau/cibil_pt_036.parquet.gzip
53692
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Bureau/cibil_pt_037.parquet.gzip
53766
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Bureau/cibil_pt_038.parquet.gzip
53844
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Bureau/cibil_pt_039.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Bureau/cibil_pt_040.parquet.gzip
/pb_analytics/

/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Internal/cibil_pt_062.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Internal/cibil_pt_063.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Internal/cibil_pt_064.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Internal/cibil_pt_065.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Internal/cibil_pt_066.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Internal/cibil_pt_067.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Internal/cibil_pt_068.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30October23/Internal/cibil_pt_069.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bure

62484
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Bureau/cibil_pt_026.parquet.gzip
62526
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Bureau/cibil_pt_027.parquet.gzip
62561
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Bureau/cibil_pt_028.parquet.gzip
62640
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Bureau/cibil_pt_029.parquet.gzip
62772
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Bureau/cibil_pt_030.parquet.gzip
63074
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Bureau/cibil_pt_031.parquet.gzip
63262
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Bureau/cibil_pt_032.parquet.gzip
63627
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Bureau/cibil_pt_033.parqu

/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Internal/cibil_pt_026.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Internal/cibil_pt_027.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Internal/cibil_pt_028.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Internal/cibil_pt_029.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Internal/cibil_pt_030.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Internal/cibil_pt_031.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Internal/cibil_pt_032.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_26September23/Internal/cibil_pt_033.parquet.gzip
/pb_analytics/analytics/

114102
/pb_analytics/analytics/acc_data/Using Hive Only/2023/08Aug/Cibil_20.parquet.gzip
114913
/pb_analytics/analytics/acc_data/Using Hive Only/2023/08Aug/Cibil_21.parquet.gzip
115251
/pb_analytics/analytics/acc_data/Using Hive Only/2023/08Aug/Cibil_22.parquet.gzip
115427
/pb_analytics/analytics/acc_data/Using Hive Only/2023/08Aug/Cibil_23.parquet.gzip
115516
/pb_analytics/analytics/acc_data/Using Hive Only/2023/08Aug/Cibil_24.parquet.gzip
115517
/pb_analytics/analytics/acc_data/Using Hive Only/2023/08Aug/Cibil_25.parquet.gzip
115517
/pb_analytics/analytics/acc_data/Using Hive Only/2023/08Aug/Cibil_26.parquet.gzip
115518
/pb_analytics/analytics/acc_data/Using Hive Only/2023/08Aug/Cibil_27.parquet.gzip
115518
/pb_analytics/analytics/acc_data/Using Hive Only/2023/08Aug/Cibil_28.parquet.gzip
115519
/pb_analytics/analytics/acc_data/Using Hive Only/2023/08Aug/Cibil_29.parquet.gzip
115520
/pb_analytics/analytics/acc_data/Using Hive Only/2023/08Aug/Cibil_30.parquet.gzip
115530
/pb_analytics/

122418
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29August23/Internal/cibil_pt_02.parquet.gzip
122418
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29August23/Internal/cibil_pt_03.parquet.gzip
122419
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29August23/Internal/cibil_pt_04.parquet.gzip
122419
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29August23/Internal/cibil_pt_05.parquet.gzip
122419
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29August23/Internal/cibil_pt_06.parquet.gzip
122419
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29August23/Internal/cibil_pt_07.parquet.gzip
122419
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29August23/Internal/cibil_pt_08.parquet.gzip
122422
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_29August23/Internal/cibil_pt_09.parquet.gzip


In [167]:
aug = pans_data
aug

,mobilePhone,applicationCreditReportId,accountHolderTypeId,createdAt,reportDate,dateClosed,openDate,dateReported,accountTypeId,accountStatusId,pincode,cityId,gender,employmentTypeId,monthlyIncome,Age,score,accountStatusName,accountTypeName,paymentHistoryProfile,highCredit,creditLimitAmount,currentBalance,amountPastDue,dateofLastPayment,panId
761,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2021-05-08,2021-04-25,2021-05-31,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648,Closed,Personal Loan,"0,0,",8374,-1,0,0,2021-05-08,NaN
762,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2020-03-31,2020-03-01,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648,Closed,Personal Loan,"0,",1000,-1,0,-1,2020-03-31,NaN
763,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2021-02-13,2020-11-16,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648,Closed,Personal Loan,"0,",1000,-1,0,-1,2021-02-13,NaN
764,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2020-07-16,2020-03-31,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648,Closed,Personal Loan,"0,",1000,-1,0,0,2020-07-16,NaN
765,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2020-07-16,2020-03-28,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648,Closed,Personal Loan,"0,",500,-1,0,0,2020-07-16,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4146439,8264826452,NaN,1,2023-05-26,2023-05-26,2014-04-22,2013-12-31,2014-04-30,7,18,391350,126.0,Male,2,30000,38.6,777.0,Closed,Gold Loan,"0,0,0,0,0",46000.0,nan,0,nan,2014-04-22,8264826452_19May2023
4146440,8264826452,NaN,1,2023-05-26,2023-05-26,2013-12-31,2012-11-26,2014-12-22,7,18,391350,126.0,Male,2,30000,38.6,777.0,Closed,Gold Loan,"0,3,0,0,0,0,0,0,0,0,0,0,0,0",53000.0,nan,0,nan,2013-07-31,8264826452_19May2023
4146441,8264826452,NaN,1,2023-05-26,2023-05-26,2017-05-28,2012-11-23,2021-03-31,6,18,391350,126.0,Male,2,30000,38.6,777.0,Closed,Consumer Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",7800.0,nan,0,nan,2017-04-11,8264826452_19May2023
4146442,8264826452,NaN,1,2023-05-26,2023-05-26,2014-05-21,2011-06-29,2014-05-31,13,18,391350,126.0,Male,2,30000,38.6,777.0,Closed,Two-Wheeler Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",43000.0,nan,0,nan,2013-07-06,8264826452_19May2023


In [11]:
total = pans_data

In [214]:
total = pd.concat([aug,jul])
total

,mobilePhone,applicationCreditReportId,accountHolderTypeId,createdAt,reportDate,dateClosed,openDate,dateReported,accountTypeId,accountStatusId,pincode,cityId,gender,employmentTypeId,monthlyIncome,Age,score,accountStatusName,accountTypeName,paymentHistoryProfile,highCredit,creditLimitAmount,currentBalance,amountPastDue,dateofLastPayment,panId
761,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2021-05-08,2021-04-25,2021-05-31,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648,Closed,Personal Loan,"0,0,",8374,-1,0,0,2021-05-08,NaN
762,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2020-03-31,2020-03-01,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648,Closed,Personal Loan,"0,",1000,-1,0,-1,2020-03-31,NaN
763,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2021-02-13,2020-11-16,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648,Closed,Personal Loan,"0,",1000,-1,0,-1,2021-02-13,NaN
764,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2020-07-16,2020-03-31,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648,Closed,Personal Loan,"0,",1000,-1,0,0,2020-07-16,NaN
765,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2020-07-16,2020-03-28,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648,Closed,Personal Loan,"0,",500,-1,0,0,2020-07-16,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4808081,7397379073,NaN,1,2023-05-26,2023-05-26,2015-01-29,2014-05-24,2021-03-31,6,18,600116,420.0,Male,1,30000,43.1,645.0,Closed,Consumer Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",15000.0,nan,0,nan,2014-12-05,7397379073_19May2023
4808082,7397379073,NaN,1,2023-05-26,2023-05-26,2014-10-11,2013-12-18,2021-03-31,6,18,600116,420.0,Male,1,30000,43.1,645.0,Closed,Consumer Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",30000.0,nan,0,nan,2014-09-02,7397379073_19May2023
4808083,7397379073,NaN,1,2023-05-26,2023-05-26,2017-08-04,2007-11-12,2017-08-31,15,18,600116,420.0,Male,1,30000,43.1,645.0,Closed,(LABD) Current Loan Against Bank Deposits,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",14000.0,nan,0,nan,NaT,7397379073_19May2023
4808084,7397379073,NaN,1,2023-05-26,2023-05-26,2014-04-30,NaT,2014-04-30,7,18,600116,420.0,Male,1,30000,43.1,645.0,Closed,Gold Loan,0,nan,nan,0,nan,NaT,7397379073_19May2023


In [13]:
total.employmentTypeId = total.employmentTypeId.astype(str)

In [14]:

total.to_parquet('cibil_base_nov_aug.parquet.gzip', index =False, compression = 'gzip')


In [212]:
total = total[['mobilePhone', 'score']]
total.drop_duplicates(subset = 'mobilePhone', keep = 'first', inplace = True)
total

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,mobilePhone,score
761,8179467226,648
1186,7877333773,693
4380,7092060007,695
4507,9896956839,580
5656,9051651247,675
...,...,...
542016,8850939643,560.0
637291,8919977616,667.0
646077,7091855954,604.0
4808042,7397379073,645.0


In [213]:
check = final[final.phone.isin(total['mobilePhone'])]
check.paid.value_counts(normalize = True)

0    0.979284
1    0.020716
Name: paid, dtype: float64

In [217]:

total['score'] = pd.to_numeric(total['score'], errors = "coerce")
aa = total.groupby('mobilePhone').agg(score = ('score','max')).reset_index()

In [201]:
check = pd.merge(check, total, left_on = 'phone', right_on = 'mobilePhone')
check['score'] = pd.to_numeric(check['score'], errors = "coerce").fillna(0).astype(int)
check.dtypes

phone           int64
Month          object
paid            int64
mobilePhone     int64
score           int64
dtype: object

In [219]:
aa.bucket = pd.cut(aa.score, bins=[-1,700,750,900],labels = ['below 700', '700-750','750+'])


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [220]:
aa.bucket.value_counts(normalize = True)

below 700    0.873850
700-750      0.081832
750+         0.044318
Name: score, dtype: float64

In [222]:
final.to_parquet('target_file.parquet.gzip', compression = 'gzip', index = False)

In [224]:
total.applicationCreditReportId = np.where(total.applicationCreditReportId.isna(),total.panId,total.applicationCreditReportId)

In [231]:
aug.applicationCreditReportId = np.where(aug.applicationCreditReportId.isna(),aug.panId,aug.applicationCreditReportId)

In [235]:
appid = total[['applicationCreditReportId']]
appid.drop_duplicates(inplace= True)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [229]:
total

,mobilePhone,applicationCreditReportId,accountHolderTypeId,createdAt,reportDate,dateClosed,openDate,dateReported,accountTypeId,accountStatusId,pincode,cityId,gender,employmentTypeId,monthlyIncome,Age,score,accountStatusName,accountTypeName,paymentHistoryProfile,highCredit,creditLimitAmount,currentBalance,amountPastDue,dateofLastPayment,panId
761,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2021-05-08,2021-04-25,2021-05-31,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648.0,Closed,Personal Loan,"0,0,",8374,-1,0,0,2021-05-08,NaN
762,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2020-03-31,2020-03-01,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648.0,Closed,Personal Loan,"0,",1000,-1,0,-1,2020-03-31,NaN
763,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2021-02-13,2020-11-16,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648.0,Closed,Personal Loan,"0,",1000,-1,0,-1,2021-02-13,NaN
764,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2020-07-16,2020-03-31,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648.0,Closed,Personal Loan,"0,",1000,-1,0,0,2020-07-16,NaN
765,8179467226,2c96a135-8b03-58a5-b6ef-bc597484f4a1,1,2023-07-01,2023-07-01,2020-07-16,2020-03-28,2021-02-28,5,18,515201.0,2.0,MALE,1.0,16500.0,27.0,648.0,Closed,Personal Loan,"0,",500,-1,0,0,2020-07-16,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4808081,7397379073,7397379073_19May2023,1,2023-05-26,2023-05-26,2015-01-29,2014-05-24,2021-03-31,6,18,600116,420.0,Male,1,30000,43.1,645.0,Closed,Consumer Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",15000.0,nan,0,nan,2014-12-05,7397379073_19May2023
4808082,7397379073,7397379073_19May2023,1,2023-05-26,2023-05-26,2014-10-11,2013-12-18,2021-03-31,6,18,600116,420.0,Male,1,30000,43.1,645.0,Closed,Consumer Loan,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",30000.0,nan,0,nan,2014-09-02,7397379073_19May2023
4808083,7397379073,7397379073_19May2023,1,2023-05-26,2023-05-26,2017-08-04,2007-11-12,2017-08-31,15,18,600116,420.0,Male,1,30000,43.1,645.0,Closed,(LABD) Current Loan Against Bank Deposits,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",14000.0,nan,0,nan,NaT,7397379073_19May2023
4808084,7397379073,7397379073_19May2023,1,2023-05-26,2023-05-26,2014-04-30,NaT,2014-04-30,7,18,600116,420.0,Male,1,30000,43.1,645.0,Closed,Gold Loan,0,nan,nan,0,nan,NaT,7397379073_19May2023


In [240]:
appid['tag'] = np.where(appid['applicationCreditReportId'].isin(aug['applicationCreditReportId']), 'Aug', 'Jul')

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [244]:
appid['tag'].value_counts()

Aug    158004
Jul     25105
Name: tag, dtype: int64

In [245]:
appid.to_parquet('enquiries.parquet.gzip',compression = 'gzip', index = False)

In [255]:
int_col = ['applicationCreditReportId','enquiryDate','enquiryType','amount','bureauDate']
scrub_col = ['applicationCreditReportId','enquiryDate','enquiryType','amount','bureauDate']
# /pb_analytics/analytics/acc_data/Using%20Hive%20Only/2022/05May
# date_list = ['Cibil_Scrub_31July23','Cibil_Scrub_30June23']
date_int = ['07Jul','Cibil_Scrub_31July23','06Jun','Cibil_Scrub_30June23','05May','Cibil_Scrub_31May23']
folder = ["/Bureau","/Internal"]
pans_data = pd.DataFrame()
mob = []
for bur in date_int:
#     if((bur == '12Dec') | (bur == '11Nov') | (bur == '10Oct') | (bur == '09Sep')):
#         year = 2022
#     else:
#         year = 2023
    print(bur)
    if bur.startswith('Cibil_Scrub'):
        print('on scrub')
        for f in folder:
            print(f)
            for i in range(1,80):
                try:
                    path = "/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/"+str(bur)+str(f)+"/enquiry_0"+str(i)+".parquet.gzip"
                    print(path)
                    dd = pd.read_parquet(path,columns=scrub_col).drop_duplicates()
                    dd = dd[dd.applicationCreditReportId.isin(appid['applicationCreditReportId'])]
                    dd = dd.loc[~dd['applicationCreditReportId'].isin(mob)]
                    mob = mob + dd['applicationCreditReportId'].to_list()
                    pans_data = pans_data.append(dd)
                    print(pans_data['applicationCreditReportId'].nunique())
                    del dd
                    gc.collect()
                except:
                    pass
    else:
        for i in range(1, 34):
            try:
                path = '/pb_analytics/analytics/acc_data/Using Hive Only/2023/'+str(bur)+'/enq_cbl_'+str(i)+'.parquet.gzip'
                #path = '/pb_analytics/analytics/acc_data/Using Hive Only/2022/'+str(q)+'/Cibil_'+str(i)+'.parquet.gzip'
                print(path)
                dd = pd.read_parquet(path, columns = int_col).drop_duplicates()
#                 dd['mobilePhone'] = pd.to_numeric(dd['mobilePhone'], errors = 'coerce')
        #                     dd['createdAt'] = pd.to_datetime(dd['createdAt'])
        #                     dd.drop_duplicates(subset = ['mobilePhone'],inplace = True)
                dd = dd[dd.applicationCreditReportId.isin(appid['applicationCreditReportId'])]
#                 dd = pd.merge(dd, disbursal, on = 'mobilePhone', how = 'inner')
#                 dd = dd[dd['reportDate']<=dd['disbursal_date']]
                dd = dd.loc[~dd['applicationCreditReportId'].isin(mob)]
                mob = mob + dd['applicationCreditReportId'].to_list()
                pans_data = pans_data.append(dd)
                print(pans_data['applicationCreditReportId'].nunique())
                del dd
                gc.collect()
            except:
                pass







07Jul
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_1.parquet.gzip
6539
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_2.parquet.gzip
11582
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_3.parquet.gzip
13700
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_4.parquet.gzip
17232
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_5.parquet.gzip
22972
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_6.parquet.gzip
29636
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_7.parquet.gzip
35624
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_8.parquet.gzip
39976
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_9.parquet.gzip
44333
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_10.parquet.gzip
49928
/pb_analytics/analytics/acc_data/Using Hive Only/2023/07Jul/enq_cbl_11.parquet.gzip
54768
/pb_analytics/

/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_059.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_060.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_061.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_062.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_063.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_064.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_065.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau/enquiry_066.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31July23/Bureau

139011
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_2.parquet.gzip
139121
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_3.parquet.gzip
139209
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_4.parquet.gzip
139261
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_5.parquet.gzip
139331
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_6.parquet.gzip
139473
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_7.parquet.gzip
139624
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_8.parquet.gzip
139883
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_9.parquet.gzip
140328
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_10.parquet.gzip
140674
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_11.parquet.gzip
141182
/pb_analytics/analytics/acc_data/Using Hive Only/2023/06Jun/enq_cbl_12.parquet.gzip
141721


/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_052.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_053.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_054.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_055.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_056.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_057.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_058.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau/enquiry_059.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Bureau

/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_047.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_048.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_049.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_050.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_051.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_052.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_053.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_30June23/Internal/enquiry_054.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub

156636
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_013.parquet.gzip
156681
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_014.parquet.gzip
156689
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_015.parquet.gzip
156694
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_016.parquet.gzip
156737
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_017.parquet.gzip
156744
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_018.parquet.gzip
156746
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_019.parquet.gzip
156751
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Bureau/enquiry_020.parquet.gzip
156807
/pb_analytics/analytics/240/Sarth

157348
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_05.parquet.gzip
157349
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_06.parquet.gzip
157351
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_07.parquet.gzip
157356
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_08.parquet.gzip
157356
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_09.parquet.gzip
157356
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_010.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_011.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_PA/External_Files/Cibil_Scrub_31May23/Internal/enquiry_012.parquet.gzip
/pb_analytics/analytics/240/Sarthak/Bureau_

In [268]:
pans_data['bureauDate'] = pd.to_datetime(pans_data['enquiryDate'])
pans_data

,applicationCreditReportId,enquiryDate,enquiryType,amount,bureauDate
13,fd56f252-7d13-5c85-b36f-2f514763738a,2022-08-04,Personal Loan,95000,2022-08-04
166,fd56f252-7d13-5c85-b36f-2f514763738a,2023-01-03,Consumer Loan,4000,2023-01-03
450,3a942953-803d-5c6a-be32-6731765432e6,2023-02-18,Personal Loan,3000,2023-02-18
497,3a942953-803d-5c6a-be32-6731765432e6,2022-04-13,Personal Loan,5000,2022-04-13
519,3a942953-803d-5c6a-be32-6731765432e6,2022-05-05,Two-Wheeler Loan,86000,2022-05-05
...,...,...,...,...,...
5905259,8264826452_19May2023,2014-10-09 00:00:00,Gold Loan,50000,2014-10-09
5905260,8264826452_19May2023,2014-05-17 00:00:00,Two-Wheeler Loan,47000,2014-05-17
5905261,8264826452_19May2023,2012-11-07 00:00:00,Consumer Loan,25000,2012-11-07
5905262,8264826452_19May2023,2011-06-15 00:00:00,Two-Wheeler Loan,43000,2011-06-15


In [271]:
pans_data['amount'] = pans_data['amount'].astype(str)

In [272]:

pans_data.to_parquet('enquiriesdump.parquet.gzip', compression = 'gzip', index = False)